In [2]:
import torch
import torch.optim as optim

1. Redefine the model to be `w2 * t_u ** 2 + w1 * t_u + b`.

    1. What parts of the training loop, and so on, need to change to accommodate this redefinition? <br>
        Only need to change model function. Rest all works.

    2. What parts are agnostic to swapping out the model? <br>
        Everything! We now have three params but rest is all the same

    3. Is the resulting loss higher or lower after training? <br>
        In my case, it's a bit higher for both train and val

    4. Is the actual result better or worse? <br>
        Not sure we never checked that even in the chapter. 
        But I'm assuming since the loss is higer so the actual result is higher too
        

In [5]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
def model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b
    

In [4]:
def loss_fn(t_p, t_c):
    return ((t_p - t_c)**2).mean()

In [18]:
def training_loop(n_epochs, optimizer, params, train_tu, val_tu, train_tc, val_tc):
    for epoch in range(1, n_epochs):
        train_tp = model(train_tu, *params)
        train_loss = loss_fn(train_tp, train_tc)

        val_tp = model(val_tu, *params)
        val_loss = loss_fn(val_tp, val_tc)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 5 or epoch % 500 == 0:
            print(f'Epoch: {epoch} \t Train: {train_loss} \t Val: {val_loss}')

    return params

In [19]:
n_val = int(0.2 * t_u.size(0))

shuffle_idx = torch.randperm(t_u.size(0))

train_idx = shuffle_idx[:-n_val]
val_idx = shuffle_idx[-n_val:]

train_idx, val_idx

(tensor([ 1,  0,  8,  4,  6,  3,  2,  9, 10]), tensor([5, 7]))

In [20]:
train_tu = t_u[train_idx]
train_tc = t_c[train_idx]

val_tu = t_u[val_idx]
val_tc = t_c[val_idx]

train_tu.shape, train_tc.shape, val_tu.shape, val_tc.shape

(torch.Size([9]), torch.Size([9]), torch.Size([2]), torch.Size([2]))

In [21]:
params = torch.tensor([1., 1., 0.,], requires_grad=True)
lr=1e-2
optimizer = optim.Adam([params], lr)

training_loop(5000, optimizer, params, train_tu, val_tu, train_tc, val_tc)

Epoch: 1 	 Train: 13626444.0 	 Val: 3083100.25
Epoch: 2 	 Train: 13354143.0 	 Val: 3021544.75
Epoch: 3 	 Train: 13084663.0 	 Val: 2960626.75
Epoch: 4 	 Train: 12818054.0 	 Val: 2900357.5
Epoch: 5 	 Train: 12554358.0 	 Val: 2840745.25
Epoch: 500 	 Train: 4.491250991821289 	 Val: 13.617194175720215
Epoch: 1000 	 Train: 4.433690547943115 	 Val: 13.434093475341797
Epoch: 1500 	 Train: 4.351741313934326 	 Val: 13.169559478759766
Epoch: 2000 	 Train: 4.2462053298950195 	 Val: 12.821548461914062
Epoch: 2500 	 Train: 4.118086814880371 	 Val: 12.386462211608887
Epoch: 3000 	 Train: 3.969865083694458 	 Val: 11.862714767456055
Epoch: 3500 	 Train: 3.8064775466918945 	 Val: 11.253778457641602
Epoch: 4000 	 Train: 3.635936737060547 	 Val: 10.571142196655273
Epoch: 4500 	 Train: 3.4692487716674805 	 Val: 9.836786270141602


tensor([-0.0470,  0.0050, -1.1289], requires_grad=True)

In [23]:
with torch.no_grad():
    tp = model(t_u, *params)

In [24]:
tp

tensor([ 3.5770, 11.8967, 13.1032, 28.6229, 12.1027,  8.5507,  3.0340,  0.2267,
         8.3305, 14.3069, 19.0927])

In [25]:
 # previous linear model result
 tp_lm = torch.tensor([ 1.4110, 12.4039, 13.6556, 26.5532, 12.6216,  8.5945,  0.4315, -6.1534, 8.3224, 14.8529, 19.2065])

In [28]:
((tp - t_c)**2).mean(), ((tp_lm-t_c)**2).mean()

(tensor(4.3672), tensor(3.0490))